In [ ]:
# get the data in as usual 
!pip install python-resize-image
from google.colab import drive
import urllib.request
import os.path
from PIL import Image
import shutil, os
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount= True)
 # %cd /content/gdrive/My\ Drive/ # need to mount to Google Drive
os.chdir("/content/gdrive/My Drive/Colab Notebooks/covid-chestxray-dataset-master/COVID19_images") ### for my testing
!pwd # should see /content/gdrive/My Drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/covid-chestxray-dataset-master/COVID19_images


## FETCHING THE DATA AND MOVING INTO FOLDERS

In [ ]:
#This function is for downloading images from a link and unzipping them to a specfied directory
def download_images(working_directory, url):

  #Working directory is where you want to files to be downloaded
  working_directory = working_directory
  #URL is the download link. **NOTE THAT THIS DOESN'T WORK WITH KAGGLE**
  url = url
  
  #Set the current working directory
  try:
    os.chdir(working_directory)
  except Exception as e:
    print(e)

  remote = urllib.request.urlopen(url)  # read remote file
  data = remote.read()  # read from remote file
  remote.close()  # close urllib request
  local = open('download.zip', 'wb')  # write binary to local file
  local.write(data)
  local.close()  # close file

  #Try to unzip the files
  try:
    with zipfile.ZipFile("download.zip", 'r') as zip_ref:
      zip_ref.extractall()
    print("Unzipping files successful!")

  except Exception as e:
    print(e)

#Example use:
#working_directory = "/content/gdrive/My Drive/Colab Notebooks/scraped"
#url = "http://images.cocodataset.org/zips/val2017.zip"
#download_images(working_directory, url)

In [ ]:
#This function is for searching the given directory and all subfolders for images and 
#placing them into one of three folders (healthy, bacterial, or covid) in the specified directory. 
#If the folders don't already exist, then they will be created.
#Image directory is the parent directory containing the images (can also be in the subfolders)
#Working directory is where you want the folders to be created and images to be moved
#Status is the type of image contained the the image_directory folder. Must be one type.

def move_images(image_directory, working_directory, status):
  image_directory = image_directory
  working_directory = working_directory
  status = status
  os.chdir(working_directory)

  dirName = 'images_directory' #### changed this 

  try:
      # Create target Directory
      os.mkdir(dirName)
      print("\nDirectory" , dirName ,  "created") 
  except FileExistsError:
      print("\nDirectory" , dirName ,  "already exists")

  folders = "healthy covid bacterial"
  folders = folders.split()
  folders = [dirName + "/" + s for s in folders]

  for names in folders:
    try:
        # Create target Directory
        os.mkdir(names)
        print("Directory" , names ,  "created ") 
    except FileExistsError:
        print("Directory" , names ,  "already exists")

  names = [os.path.join(name)
             for root, dirs, files in os.walk(image_directory)
             for name in files
             if name.endswith((".jpg", ".jpeg", ".gif", ".png", ".tif", ".bmp"))]

  roots = [os.path.join(root)
              for root, dirs, files in os.walk(image_directory)
              for name in files
              if name.endswith((".jpg", ".jpeg", ".gif", ".png", ".tif", ".bmp"))]

  filepath = os.path.join(working_directory, "images_directory", status)

  for root, name in zip(roots, names):
    if os.path.exists(filepath + name):
      pass
    else:
      shutil.copy(os.path.join(root, name), filepath)

#Example use:
#image_directory = "/content/gdrive/My Drive/Colab Notebooks/scraped/covid-chestxray-dataset-master/images"
#working_directory = "/content/gdrive/My Drive/Colab Notebooks/scraped"
#status = "bacterial"

#move_images(image_directory, working_directory, status)

In [ ]:
#This function is for cropping images from the center
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

In [ ]:
#This function will take process all images in the specified folder, including subfolders. 
#We can play with the resizing parameters
def image_preprocessing(image_directory):

  imagefiles = [os.path.join(root, name)
             for root, dirs, files in os.walk(image_directory)
             for name in files
             if name.endswith((".jpg", ".jpeg", ".gif", ".png", ".tif", ".bmp"))]

  filepath = os.path.join(working_directory, "images_directory", status)

  for item in imagefiles:
      im = Image.open(item)
      f, e = os.path.splitext(item)
      im = im.resize((1500,1500), Image.ANTIALIAS)
      im = crop_center(im, 1200, 1200)
      im = im.resize((224,224), Image.ANTIALIAS)
      im.save(item, format = "PNG", quality=90)

#image_directory = "/content/gdrive/My Drive/Colab Notebooks/scraped/images_directory/healthy"
#image_preprocessing(image_directory)

In [ ]:
working_directory = "/content/gdrive/My Drive/Colab Notebooks/scraped"
url = "https://github.com/ieee8023/covid-chestxray-dataset/archive/master.zip"
download_images(working_directory, url)

Unzipping files successful!


In [ ]:
image_directory = "/content/gdrive/My Drive/Colab Notebooks/scraped/covid-chestxray-dataset-master/images"
working_directory = "/content/gdrive/My Drive/Colab Notebooks/scraped"
status = "covid"

move_images(image_directory, working_directory, status)


Directory images_directory already exists
Directory images_directory/healthy already exists
Directory images_directory/covid already exists
Directory images_directory/bacterial already exists


In [ ]:
image_directory = "/content/gdrive/My Drive/Colab Notebooks/scraped/images_directory"
image_preprocessing(image_directory)